In [8]:
!pip install streamlit pyngrok pandas scikit-learn requests plotly > /dev/null 2>&1

import os

OPENWEATHER_API_KEY = os.environ.get("OPENWEATHER_API_KEY")
NGROK_AUTH_TOKEN = os.environ.get("NGROK_AUTH_TOKEN")

app_code = f"""
import streamlit as st
import pandas as pd
import requests
from datetime import datetime
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import numpy as np
import os
import time

API_KEY = os.getenv("OPENWEATHER_API_KEY")

st.set_page_config(page_title="Live Weather Dashboard", layout="wide")
st.title("🌦 Live Weather Monitoring & Prediction")

city = st.text_input("Enter city name (e.g., Delhi,IN):", "Delhi,IN")
time_frame = st.number_input("Enter refresh interval (in minutes):", min_value=1, value=5)

URL = f"https://api.openweathermap.org/data/2.5/weather?q={{city}}&appid={{API_KEY}}&units=metric"

@st.cache_data(ttl=60)
def fetch_weather():
    try:
        r = requests.get(URL)
        data = r.json()
        if "main" in data:
            return {{
                "time": datetime.now(),
                "temp": data['main']['temp'],
                "humidity": data['main']['humidity']
            }}
        else:
            st.error(f"API Error: {{data}}")
            return None
    except Exception as e:
        st.error(f"Error fetching weather: {{e}}")
        return None

if "weather_data" not in st.session_state:
    st.session_state.weather_data = []

# 🔄 Auto-refresh simulation (instead of experimental_autorefresh)
placeholder = st.empty()
if st.button("🔄 Refresh Data"):
    new_data = fetch_weather()
    if new_data:
        st.session_state.weather_data.append(new_data)

df = pd.DataFrame(st.session_state.weather_data)

if not df.empty:
    col1, col2 = st.columns(2)

    with col1:
        st.subheader("🌡️ Temperature Over Time")
        fig_temp = go.Figure()
        fig_temp.add_trace(go.Scatter(x=df["time"], y=df["temp"], mode="lines+markers", line=dict(shape="spline")))
        fig_temp.update_layout(title="Temperature (°C)")
        st.plotly_chart(fig_temp, use_container_width=True)

    with col2:
        st.subheader("💧 Humidity Over Time")
        fig_hum = go.Figure()
        fig_hum.add_trace(go.Scatter(x=df["time"], y=df["humidity"], mode="lines+markers", line=dict(shape="spline")))
        fig_hum.update_layout(title="Humidity (%)")
        st.plotly_chart(fig_hum, use_container_width=True)

    st.subheader("📈 Next-Day Prediction")
    df["day_num"] = np.arange(len(df))
    model_temp = LinearRegression()
    model_hum = LinearRegression()
    model_temp.fit(df[["day_num"]], df["temp"])
    model_hum.fit(df[["day_num"]], df["humidity"])

    next_day = [[len(df)]]
    st.metric("Predicted Temp (°C)", f"{{model_temp.predict(next_day)[0]:.2f}}")
    st.metric("Predicted Humidity (%)", f"{{model_hum.predict(next_day)[0]:.2f}}")

else:
    st.warning("⚠️ No weather data yet. Click 'Refresh Data' to fetch the first record.")

st.info(f"Weather data can be refreshed every {{time_frame}} minutes manually.")
"""

with open("app.py", "w") as f:
    f.write(app_code)

from pyngrok import ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(8501)
print("🌍 Public URL:", public_url)

!streamlit run app.py --server.port 8501 & sleep 3


🌍 Public URL: NgrokTunnel: "https://dfe80dcf3fb3.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.10.251:8501

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

  Stopping...
